In [30]:
#!pip install langchain_openai
import os
from langchain.tools import tool
from serpapi import search
from typing import List
from dotenv import load_dotenv

load_dotenv()

serp_apikey= os.getenv("SERP_API_KEY")
tavily_api_key= os.getenv("TAVILY_API_KEY")
open_weather_api_key = os.getenv("OPEN_WEATHER_API_KEY")

from tavily import TavilyClient

from typing import TypedDict, Annotated, List, Dict, Any
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
import operator
from langchain_groq import ChatGroq
api_key = os.getenv("GROQ_API_KEY")
from langgraph.graph import StateGraph, MessagesState, END
from typing import Literal
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode
from langchain_core.runnables import RunnableConfig

In [31]:
def get_hotel_details(dictionary):
    name = dictionary['name']
    rate = dictionary['rate_per_night'] if 'rate_per_night' in dictionary else 'NA'
    overall_rating = dictionary['overall_rating'] if 'overall_rating' in dictionary else 'NA'
    return {'property_name' : name, 'rate_per_night': rate, 'rating' : overall_rating}

In [32]:
def get_flight_details(dictionary):
    airlines = [flight['airline'] for flight in dictionary['flights']]
    layover_stations = dictionary['layovers'] if 'layovers' in dictionary.keys() else 'NA'
    total_duration = dictionary['total_duration'] 
    price = dictionary['price'] if 'price' in dictionary.keys() else 'Not Provided'
    return {'airlines': airlines,
            'layover': layover_stations,
            'Duration': total_duration,
            'price': price}

In [33]:
@tool('hotel-search')
def hotel_search(place : str, 
                 check_in_date : str, 
                 check_out_date : str, 
                 country_location :str, 
                 number_of_adults : int,
                 number_of_children : int = 0,
                 children_ages : List[int] = None,
                 property_types : int = None,
                 min_price : int = None,
                 max_price : int = None):
    
    '''
    Searches for hotels at the destination location. Use first two letter for country location eg., in, uk, us
    Returns: It returns a dict containing the following -
             place: Location of the properties, property_name : Name of the property, rate : rate per night in INR, rating : overall rating by the customers
    '''
    params = {
        "engine": "google_hotels",
        "q": place,
        "check_in_date": check_in_date,
        "check_out_date":check_out_date,
        "adults": number_of_adults,
        "children": number_of_children,
        "children_ages" : children_ages,
        "property_types" : property_types,
        "currency": "INR",
        "gl": country_location,
        "hl": "en",
        "max_price" : max_price,
        "min_price" : min_price,
        "api_key": serp_apikey
    }

    results = search(params)
    return_dict = list(map(get_hotel_details, results['properties']))[:4]
    return {'place' : place, 'hotel_details': return_dict}

In [34]:
# # Basic test call
# result = hotel_search(
#     place="Mumbai",
#     check_in_date="2025-07-15",
#     check_out_date="2025-07-18",
#     country_location="in",
#     number_of_adults=2
# )
# print(result)

In [35]:

@tool('get_attraction')
def _get_attraction(city: str):
    '''
    Searches for attractions in the specified city using Tavily API
    
    Args:
        city (str): Name of the city to search attractions for
        
    Returns:
        dict: Contains city name and list of attraction details with name, description, and rating
    '''
    
    # Initialize Tavily client
    tavily_client = TavilyClient(api_key=tavily_api_key)
    
    # Search query for attractions
    query = f"top attractions tourist places things to do in {city}"
    
    try:
        # Search using Tavily
        response = tavily_client.search(
            query=query,
            search_depth="advanced",
            max_results=5,
            include_domains=["tripadvisor.com", "lonelyplanet.com", "timeout.com", "google.com"]
        )
        
        # Extract and format attraction details
        attractions = []
        for result in response.get('results', [])[:6]:  # Limit to top 6 attractions
            attraction = {
                'name': result.get('title', 'N/A'),
                'description': result.get('content', 'N/A')[:200] + '...' if len(result.get('content', '')) > 200 else result.get('content', 'N/A'),
                'url': result.get('url', 'N/A'),
                'score': result.get('score', 0)
            }
            attractions.append(attraction)
        
        return {
            'city': city,
            'attractions': attractions,
            'total_found': len(attractions)
        }
        
    except Exception as e:
        return {
            'city': city,
            'attractions': [],
            'error': f"Error fetching attractions: {str(e)}"
        }





In [36]:
@tool('search_restaurants')
def search_restaurants(city: str):
    '''Search for restaurants and food options in the city'''
    tavily_client = TavilyClient(api_key=tavily_api_key)
    query = f"best restaurants food places to eat in {city}"
    
    try:
        response = tavily_client.search(
            query=query,
            search_depth="advanced",
            max_results=5,
            include_domains=["zomato.com", "tripadvisor.com", "timeout.com", "lonelyplanet.com"]
        )
        
        restaurants = []
        for result in response.get('results', [])[:5]:
            restaurant = {
                'name': result.get('title', 'N/A'),
                'description': result.get('content', 'N/A')[:200] + '...' if len(result.get('content', '')) > 200 else result.get('content', 'N/A'),
                'url': result.get('url', 'N/A'),
                'score': result.get('score', 0)
            }
            restaurants.append(restaurant)
        
        return {
            'city': city,
            'restaurants': restaurants,
            'total_found': len(restaurants)
        }
    except Exception as e:
        return {
            'city': city,
            'restaurants': [],
            'error': f"Error fetching restaurants: {str(e)}"
        }

In [37]:
# result1 = _get_attraction("Mumbai")
# result1


In [38]:

@tool('search_activities')
def search_activities(city: str):
    '''Search for activities and experiences in the city'''
    tavily_client = TavilyClient(api_key=tavily_api_key)
    query = f"activities experiences tours things to do {city}"
         
    try:
        response = tavily_client.search(
            query=query,
            search_depth="advanced",
            max_results=5
        )
                 
        activities = []
        for result in response.get('results', [])[:5]:
            activities.append(result.get('title', 'N/A'))
                 
        return activities
    except Exception as e:
        return [f"Error fetching activities: {str(e)}"]

In [39]:
print(search_activities("Paris"))

['THE 15 BEST Things to Do in Paris (2025) - Must-See Attractions', 'Unique experiences for 4 days in Paris, please? - Rick Steves Travel ...', '204 Cool and Unusual Things to Do in Paris - Atlas Obscura', 'Book attractions, activities, tours and transportation in Paris', 'The Best Tours & Experiences in Paris - Civitatis']


In [40]:
#@tool('flight-search')
def flight_search(
        departure_id: str,
        arrival_id: str,
        outbound_date: str,
        return_date: str = None,
        currency: str = "INR",
    ):
    
    '''
    Searches for flights based on the provided criteria using a flight search engine. Provide IATA ids for the locations.
    Returns:
        dict: A dictionary containing the search results from the flight search engine. 
              airlines : List of airlines at each point departure to layovers to arrival 
              layover : list of layovers, NA if direct flight
              Duration : Total duration in minutes
              price : price in INR
    '''
    params = {
    "engine": "google_flights",
    "departure_id": departure_id,
    "arrival_id": arrival_id,
    "outbound_date": outbound_date,
    "return_date": return_date,
    "currency": currency,
    "hl": "en",
    "type": 2 if return_date == None else 1,
    "api_key": os.getenv('SERP_API_KEY')
    }

    results = search(params)
    return_dict = list(map(get_flight_details, dict(results)['other_flights']))
    return {'Departure': departure_id, 'Arrival':arrival_id, 'flight_details':return_dict}

In [41]:
# print("=== Test 2: Round-trip Domestic Flight ===")
# result2 = flight_search(
#     departure_id="BLR",  # Bangalore
#     arrival_id="CCU",    # Kolkata
#     outbound_date="2025-08-01",
#     return_date="2025-08-05",
#     currency="INR"
# )
# result2

In [42]:
import requests
import os
from datetime import datetime, timedelta
from typing import Dict, Any

@tool
def get_weather_forecast(city: str, date: str) -> Dict[str, Any]:
    '''
    Weather forecast tool for travel agent - gets weather information for travel dates
    
    Args:
        city (str): City name extracted from other travel tools (e.g., "Mumbai", "Paris", "New York")
        date (str): Travel date in YYYY-MM-DD format from travel itinerary
        
    Returns:
        dict: Weather information for travel planning containing:
            - status: "success" or "error"
            - city: Normalized city name
            - date: Requested date
            - temperature: min, max, current temperature in Celsius
            - conditions: Weather description
            - humidity: Humidity percentage
            - wind_speed: Wind speed in m/s
            - rain_probability: Chance of rain (0-100%)
    '''
    
    try:
        # Validate date format and check forecast availability
        target_date = datetime.strptime(date, "%Y-%m-%d")
        today = datetime.now().date()
        days_difference = (target_date.date() - today).days
        
        # Check forecast range
        if days_difference < 0:
            return {
                "status": "error",
                "city": city,
                "date": date,
                "message": "Cannot get weather for past dates"
            }
        elif days_difference > 5:
            return {
                "status": "error", 
                "city": city,
                "date": date,
                "message": "Weather forecast available only for next 5 days"
            }
        
        # Get weather data
        weather_data = _fetch_openweather_data(city, date, days_difference)
        
        if weather_data.get("error"):
            return {
                "status": "error",
                "city": city, 
                "date": date,
                "message": weather_data["error"]
            }
        
        return weather_data
        
    except ValueError:
        return {
            "status": "error",
            "city": city,
            "date": date,
            "message": "Invalid date format. Use YYYY-MM-DD"
        }
    except Exception as e:
        return {
            "status": "error",
            "city": city,
            "date": date, 
            "message": f"Weather service error: {str(e)}"
        }

def _fetch_openweather_data(city: str, date: str, days_diff: int) -> Dict[str, Any]:
    '''Fetch weather data from OpenWeatherMap API'''
    
    api_key = open_weather_api_key
    print(api_key)
    if not api_key:
        return {"error": "Weather API key not configured"}
    
    try:
        # Get city coordinates
        lat, lon, normalized_city = _get_city_coordinates(city, api_key)
        if not lat:
            return {"error": f"City '{city}' not found"}
        
        # Get weather data based on date
        if days_diff == 0:
            return _get_current_weather(lat, lon, normalized_city, date, api_key)
        else:
            return _get_forecast_weather(lat, lon, normalized_city, date, api_key)
            
    except requests.exceptions.RequestException:
        return {"error": "Weather service connection failed"}
    except Exception as e:
        return {"error": f"Failed to get weather data: {str(e)}"}

def _get_city_coordinates(city: str, api_key: str) -> tuple:
    '''Get coordinates for city'''
    
    # Clean city name
    clean_city = city.strip().split(',')[0]
    
    geocoding_url = "http://api.openweathermap.org/geo/1.0/direct"
    params = {
        "q": clean_city,
        "limit": 1,
        "appid": api_key
    }
    
    response = requests.get(geocoding_url, params=params, timeout=10)
    response.raise_for_status()
    
    geo_data = response.json()
    if not geo_data:
        return None, None, None
    
    location_info = geo_data[0]
    lat = location_info['lat']
    lon = location_info['lon']
    normalized_city = f"{location_info['name']}, {location_info['country']}"
    
    return lat, lon, normalized_city

def _get_current_weather(lat: float, lon: float, city: str, date: str, api_key: str) -> Dict[str, Any]:
    '''Get current weather'''
    
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "lat": lat,
        "lon": lon,
        "appid": api_key,
        "units": "metric"
    }
    
    response = requests.get(url, params=params, timeout=10)
    response.raise_for_status()
    data = response.json()
    
    return {
        "status": "success",
        "city": city,
        "date": date,
        "temperature": {
            "min": round(data['main']['temp_min']),
            "max": round(data['main']['temp_max']),
            "current": round(data['main']['temp'])
        },
        "conditions": data['weather'][0]['description'].title(),
        "humidity": data['main']['humidity'],
        "wind_speed": round(data['wind']['speed'], 1),
        "rain_probability": 0,  # Current weather doesn't have probability
        "data_type": "current"
    }

def _get_forecast_weather(lat: float, lon: float, city: str, date: str, api_key: str) -> Dict[str, Any]:
    '''Get forecast weather'''
    
    url = "https://api.openweathermap.org/data/2.5/forecast"
    params = {
        "lat": lat,
        "lon": lon,
        "appid": api_key,
        "units": "metric"
    }
    
    response = requests.get(url, params=params, timeout=10)
    response.raise_for_status()
    data = response.json()
    
    # Find forecasts for the target date
    target_forecasts = []
    for item in data['list']:
        forecast_datetime = datetime.fromtimestamp(item['dt'])
        if forecast_datetime.strftime('%Y-%m-%d') == date:
            target_forecasts.append(item)
    
    if not target_forecasts:
        return {"error": f"No forecast data available for {date}"}
    
    # Get midday forecast for representative conditions
    midday_forecast = min(target_forecasts, 
                         key=lambda x: abs(datetime.fromtimestamp(x['dt']).hour - 12))
    
    # Calculate temperature range for the day
    day_temps = [item['main']['temp'] for item in target_forecasts]
    
    return {
        "status": "success",
        "city": city,
        "date": date,
        "temperature": {
            "min": round(min(day_temps)),
            "max": round(max(day_temps)),
            "current": round(midday_forecast['main']['temp'])
        },
        "conditions": midday_forecast['weather'][0]['description'].title(),
        "humidity": midday_forecast['main']['humidity'],
        "wind_speed": round(midday_forecast['wind']['speed'], 1),
        "rain_probability": round(midday_forecast.get('pop', 0) * 100),
        "data_type": "forecast"
    }

In [43]:
# weather = get_weather_forecast("Delhi", "2025-06-23")
# print("Weather in Singapore:", weather)

In [44]:

@tool
def add_costs(cost1: float, cost2: float) -> float:
    """Add two costs together"""
    return cost1 + cost2

@tool
def multiply_costs(cost: float, multiplier: float) -> float:
    """Multiply cost by a multiplier (for multi-day/multi-person)"""
    return cost * multiplier

@tool('calculate_total_expense')
def calculate_total_expense(costs: list):
    '''Calculate total expense from a list of costs'''
    try:
        total = sum(float(cost) for cost in costs if cost)
        return f"Total expense: {total:.2f}"
    except (ValueError, TypeError) as e:
        return f"Error calculating total: {str(e)}"

@tool
def calculate_daily_budget(total_cost: float, days: int) -> float:
    """Calculate daily budget breakdown"""
    return total_cost / days if days > 0 else 0

In [45]:

@tool
def estimate_hotel_cost(price_per_night: float, total_nights: int) -> float:
    """Calculate total hotel cost for the stay"""
    return price_per_night * total_nights

In [46]:
@tool
def get_budget_range(total_budget: float) -> Dict[str, float]:
    """Get budget ranges for different categories"""
    return {
        "accommodation": total_budget * 0.40,  # 40% for hotels
        "food": total_budget * 0.25,          # 25% for food
        "transport": total_budget * 0.15,     # 15% for transport
        "activities": total_budget * 0.15,    # 15% for activities
        "miscellaneous": total_budget * 0.05  # 5% for misc
    }

In [47]:
@tool('get_transport_pricing')
def _get_transport_pricing(city: str):
    '''Search for transportation pricing in the city'''
    tavily_client = TavilyClient(api_key=tavily_api_key)
    query = f"{city} public transport prices fares day pass weekly pass metro bus train tickets cost"
    
    try:
        response = tavily_client.search(
            query=query,
            search_depth="advanced",
            max_results=6,
            include_domains=["official-city-sites.com", "rome2rio.com", "timeout.com", "tripadvisor.com"]
        )
        
        pricing_info = []
        for result in response.get('results', [])[:6]:
            pricing = {
                'title': result.get('title', 'N/A'),
                'pricing_details': result.get('content', 'N/A')[:300] + '...' if len(result.get('content', '')) > 300 else result.get('content', 'N/A'),
                'url': result.get('url', 'N/A'),
                'score': result.get('score', 0)
            }
            pricing_info.append(pricing)
        
        return {
            'city': city,
            'pricing_info': pricing_info,
            'total_found': len(pricing_info)
        }
    except Exception as e:
        return {
            'city': city,
            'pricing_info': [],
            'error': f"Error fetching transport pricing: {str(e)}"
        }
    




In [48]:
@tool('get_transport_pricing')
def _get_transport_pricing(city: str):
    '''
    Searches for transportation pricing, passes, and fare information in the specified city
    
    Args:
        city (str): Name of the city to search transport pricing for
        
    Returns:
        dict: Contains city name and pricing details for various transport options
    '''
    
    # Initialize Tavily client
    tavily_client = TavilyClient(api_key=tavily_api_key)
    
    # Search query for transport pricing
    query = f"{city} public transport prices fares day pass weekly pass metro bus train tickets cost"
    
    try:
        # Search using Tavily
        response = tavily_client.search(
            query=query,
            search_depth="advanced",
            max_results=6,
            include_domains=["official-city-sites.com", "rome2rio.com", "timeout.com", "tripadvisor.com", "lonelyplanet.com", "budgetyourtrip.com"]
        )
        
        # Extract and format pricing details
        pricing_info = []
        for result in response.get('results', [])[:6]:
            pricing = {
                'title': result.get('title', 'N/A'),
                'pricing_details': result.get('content', 'N/A')[:300] + '...' if len(result.get('content', '')) > 300 else result.get('content', 'N/A'),
                'url': result.get('url', 'N/A'),
                'score': result.get('score', 0),
                'source': result.get('url', '').split('/')[2] if result.get('url') else 'N/A'
            }
            pricing_info.append(pricing)
        
        return {
            'city': city,
            'pricing_info': pricing_info,
            'total_found': len(pricing_info)
        }
        
    except Exception as e:
        return {
            'city': city,
            'pricing_info': [],
            'error': f"Error fetching transport pricing: {str(e)}"
        }

In [49]:
# _get_transport_pricing("Paris")

In [50]:
@tool
def create_day_plan(city: str, day_number: int, attractions: str, weather: str) -> str:
    """Create a detailed day plan for the trip"""
    return f"""Day {day_number} in {city}:
Weather: {weather}
Recommended activities: {attractions[:200]}...
Tips: Plan indoor activities if weather is poor."""

In [51]:
@tool
def create_full_itinerary(city: str, total_days: int, attractions: List[str], 
                         weather_info: str, activities: List[str]) -> str:
    """Create complete itinerary for the entire trip"""
    itinerary = f"Complete {total_days}-Day Itinerary for {city}\n"
    itinerary += f"Weather Overview: {weather_info}\n\n"
    
    for day in range(1, total_days + 1):
        itinerary += f"Day {day}:\n"
        if day <= len(attractions):
            itinerary += f"- Main Attraction: {attractions[day-1]}\n"
        if day <= len(activities):
            itinerary += f"- Activity: {activities[day-1]}\n"
        itinerary += "\n"
    
    return itinerary
   

In [52]:

@tool
def create_trip_summary(destination: str, duration: int, total_cost: float, 
                       daily_budget: float, highlights: List[str]) -> str:
    """Create comprehensive trip summary"""
    summary = f"""
TRIP SUMMARY
============
Destination: {destination}
Duration: {duration} days
Total Cost: ₹{total_cost:,.2f}
Daily Budget: ₹{daily_budget:,.2f}

Highlights:
"""
    for highlight in highlights[:5]:
        summary += f"- {highlight}\n"
    
    return summary

In [53]:
tools = [
    # Step 1: Attractions and Activities
    _get_attraction, search_restaurants, search_activities, _get_transport_pricing,
    
    # Step 2: Weather
    get_weather_forecast,
    
    # Step 3: Hotels
    hotel_search, estimate_hotel_cost, get_budget_range,
    
    # Step 4: Cost Calculations
    add_costs, multiply_costs, calculate_total_expense, calculate_daily_budget,
   
    # Step 5: Itinerary
    create_day_plan, create_full_itinerary,
    
    # Step 6: Summary
    create_trip_summary
]


In [54]:
TRAVEL_AGENT_SYSTEM_PROMPT = """
You are an expert AI Travel Agent following a structured 7-step workflow for comprehensive trip planning.

MANDATORY WORKFLOW - Follow these steps in order:

STEP 1: SEARCH ATTRACTIONS AND ACTIVITIES
- Use _get_attraction to find top attractions
- Use search_restaurants to find dining options  
- Use search_activities to find experiences
- Use _get_transport_pricing for transport costs

STEP 2: WEATHER FORECASTING
- Use get_weather_forecast for current weather and forecast

STEP 3: HOTEL SEARCH AND COSTS
- Use hotel_search to find accommodations
- Use estimate_hotel_cost to calculate hotel expenses
- Use get_budget_range to allocate budget categories

STEP 4: CALCULATE TOTAL COSTS (MANDATORY - USE ALL TOOLS)
- Use add_costs to combine expense categories
- Use multiply_costs for multi-day/multi-person calculations  
- Use calculate_total_expense for final totals
- Use calculate_daily_budget for daily breakdowns


STEP 5: ITINERARY GENERATION  
- Use create_day_plan for daily plans
- Use create_full_itinerary for complete schedule

STEP 6: CREATE TRIP SUMMARY
- Use create_trip_summary for final comprehensive summary

CRITICAL COST CALCULATION RULES:
1. NEVER manually calculate costs - always use the cost calculation tools
2. You MUST use all 4 cost calculation tools: add_costs, multiply_costs, calculate_total_expense, calculate_daily_budget
3. Show step-by-step tool usage for transparency
4. Extract numeric values from pricing data before using cost tools

WORKFLOW EXECUTION:
- Complete each step before moving to the next
- Use multiple tools per step when needed
- Show progress through each step clearly
- Always end with a comprehensive trip summary

Remember: The cost calculation tools are MANDATORY - never skip Step 4!
"""

In [59]:
# Initialize LLM
llm = ChatOpenAI(
            model="gpt-4.1-mini",
            openai_api_key=os.getenv("OPENAI_API_KEY")
            #openai_api_base=os.getenv("BASE_URL"),
        )

# llm.invoke("what is AI")

In [60]:
llm_with_tools = llm.bind_tools(tools)
# Create tool node
tool_node = ToolNode(tools)


In [62]:
def travel_planner_agent(state: MessagesState):
    """Simplified travel planning agent"""
    messages = state["messages"]
    
    # Add system message if not present
    if not messages or not isinstance(messages[0], SystemMessage):
        system_message = SystemMessage(content=TRAVEL_AGENT_SYSTEM_PROMPT)
        messages = [system_message] + messages
    
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "end"]:
    """Simple continuation logic"""
    messages = state["messages"]
    last_message = messages[-1]
    
    # Continue if there are tool calls to execute
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    
    return "end"

# Create simplified workflow
workflow = StateGraph(MessagesState)
workflow.add_node("agent", travel_planner_agent)
workflow.add_node("tools", tool_node)
workflow.set_entry_point("agent")
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", "end": END})
workflow.add_edge("tools", "agent")

travel_agent = workflow.compile()

def run_travel_agent(user_request: str):
    """Run the simplified travel planning agent"""
    config = RunnableConfig(recursion_limit=40)  # Reduced limit
    initial_state = {"messages": [HumanMessage(content=user_request)]}

    for event in travel_agent.stream(initial_state, config=config):
        for value in event.values():
            if "messages" in value:
                last_message = value["messages"][-1]
                if hasattr(last_message, 'content') and last_message.content:
                    print("Agent:", last_message.content)
                if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
                    print(f"Tools: {[tc['name'] for tc in last_message.tool_calls]}")

In [64]:
# Example usage
if __name__ == "__main__":
    user_input = """
    I want to plan a 5-day trip to Delhi for 2 people in August 2025 from 1st August to 5th August. 
    Budget is around 5 lac. Looking for cultural attractions, good food, 
    and efficient local transport options.
    """
    
    run_travel_agent(user_input)

Tools: ['get_attraction', 'search_restaurants', 'search_activities', 'get_transport_pricing']
Agent: {"city": "Delhi", "pricing_info": [{"title": "New Delhi (Station) to Delhi Public School, Mathura Road", "pricing_details": "Delhi Transport Corporation (DTC) operates a bus from New Delhi Railway Station Gate 2 to Delhi Public School every 20 minutes. Tickets cost ₹9–35 and the", "url": "https://www.rome2rio.com/s/New-Delhi-Station/Delhi-Public-School-Mathura-Road", "score": 0.61307496, "source": "www.rome2rio.com"}, {"title": "Rajiv chowk to Delhi - 4 ways to travel via subway, bus, taxi ...", "pricing_details": "Delhi Metro operates a subway from Rajiv Chowk to Chawri Bazar every 10 minutes. Tickets cost ₹20–27 and the journey takes 5 min. Alternatively, Delhi Transport", "url": "https://www.rome2rio.com/s/Rajiv-Chowk/Delhi", "score": 0.57035583, "source": "www.rome2rio.com"}, {"title": "Delhi (Station) to Gurdwara Bangla Sahib - 6 ways to travel ...", "pricing_details": "Delhi Metro